# Drizzled PSFs from _HST_/ACS Imaging

This example notebook shows how `spike` interacts with _HST_/ACS imaging to generate a drizzled PSF. This notebook uses observations from [this program](https://archive.stsci.edu/cgi-bin/mastpreview?mission=hst&amp;dataid=J8PU42010). The easiest way to find these data in the MAST archive is by searching for observation ID j8pu42010. `spike` uses calibrated, but not-yet co-added images (i.e., "Level 2" data products), so be sure that your download includes 'flc' files. If you download all of the files associated with this program, your working directory will include:
- j8pu42ecq_flc.fits
- j8pu42egq_flc.fits
- j8pu42esq_flc.fits
- j8pu42evq_flc.fits

The principle of this example notebook is the same regardless of dataset, though, so it can be used as a direct guide for use with other data, too.

_NOTE: Environment variables are not always read in properly within Jupyter notebooks/JupyterLab -- see [here](https://github.com/avapolzin/spike/blob/master/example_notebooks/README.md) for some additional details of how to access them._

In [1]:
from spike.psf import hst # import the relevant top-level module from spike

datapath = '/Users/avapolzin/Desktop/psf_testoutputs/acs' #'/path/to/acs/data'
outputpath = 'psfs' #default is /psfs, defined from your working directory

Note that if you choose to create drizzled PSFs for the same data and object location using multiple PSF generation methods, you will want to define a new 'savedir' each time to avoid conflicts between files with the same name.

We'll also define an object -- in this case, a set of coordinates -- for which the PSF will be generated.

In [3]:
obj = '10:00:33.0178 +02:09:52.304' #can also be a resolvable name or coordinates in decimal degress

Now, we can call `spike.psf.hst` -- as an initial example we'll use the most minimal inputs. By default, in addition to saving drizzled PSFs and any intermediate products in the directory given by 'savedir' (here the argument will be the outputpath variable we definied), this function will return a dictionary that stores model PSFs indexed by filter and object. 

If `returnpsf = 'full'` (default), the arrays in the dictionary will be the full drizzled image field of view. If `returnpsf = 'crop'`, the arrays in the dictionary will be the cutout region around the object location with the size of the cutout determined by the 'cutout_fov' argument; the cropped region around the drizzled PSF will also be saved as a FITS file, including its WCS information (can be toggled off with `savecutout = False`).

`spike` assumes that the input files have not yet been "tweaked" and handles that step for you. If your input images have been tweaked _or_ if you would like to skip this step for other reasons, specify `pretweaked = True` in calling `spike.psf.hst`.

In [4]:
dpsf = hst(img_dir = datapath, obj = obj, img_type = 'flc', inst = 'ACS', camera = 'WFC', savedir = outputpath, pretweaked = True)



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    


2025-05-07 19:36:37,408 - stpipe - WARNING - /Users/avapolzin/opt/anaconda3/envs/spike/lib/python3.10/site-packages/spike/psf/psf.py:185: Warning: All of major, minor, and angle must be specified to be applied. Proceeding with no elliptical jitter.



The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol
Tiny Tim v7.5
Intermediate PSF dimensions are 464 by 464

Computing PSF for position 1/1 (x,y) = 1020 181
   Computing PSF 1/18 for wavelength 393.82 nm (weight=0.008917)
   Computing PSF 2/18 for wavelength 402.92 nm (weight=0.024755)
   Computing PSF 3/18 for wavelength 412.02 nm (weight=0.037561)
   Computing PSF 4/18 for wavelength 421.11 nm (weight=0.045079)
   Computing PSF 5/18 for wavelength 430.21 nm (weight=0.052658)
   Computing PSF 6/18 for wavelength 439.31 nm (weight=0.056584)
   Computing PSF 7/18 for wavelength 448.41 nm (weight=0.059373)
   Computing PSF 8/18 for wavelength 457.51 nm (weight=0.061140)
   Comp

2025-05-07 19:36:40,946 - stpipe - WARNING - /Users/avapolzin/opt/anaconda3/envs/spike/lib/python3.10/site-packages/spike/psf/psf.py:185: Warning: All of major, minor, and angle must be specified to be applied. Proceeding with no elliptical jitter.



Tiny Tim v7.5
Processing PSF for position 1/1 : (x,y) = 1020 181
Reading input PSF from /Users/avapolzin/Desktop/psf_testoutputs/acs/j8pu42ecq_flc150d08m15.267s+2d09m52.304s_F475W_psf00_psf.fits.
  Input critically-sampled undistorted PSF dimensions are 464 by 464 (0.013018 arcsec/pixel).
  Mapping PSF onto distorted grid.
  Convolving PSF with charge diffusion kernel.
  Writing distorted PSF to /Users/avapolzin/Desktop/psf_testoutputs/acs/j8pu42ecq_flc150d08m15.267s+2d09m52.304s_F475W_psf00.fits (146 by 146 pixels)

Started at  Wed May  7 19:36:40 2025
Finished at Wed May  7 19:36:40 2025
Tiny Tim v7.5
Intermediate PSF dimensions are 464 by 464

Computing PSF for position 1/1 (x,y) = 1025 242
   Computing PSF 1/18 for wavelength 393.82 nm (weight=0.008917)
   Computing PSF 2/18 for wavelength 402.92 nm (weight=0.024755)
   Computing PSF 3/18 for wavelength 412.02 nm (weight=0.037561)
   Computing PSF 4/18 for wavelength 421.11 nm (weight=0.045079)
   Computing PSF 5/18 for wavelength 

...

                  Total          5.2300 sec.


2025-05-07 19:36:55,894 - stpipe - INFO - 


2025-05-07 19:36:55,900 - stpipe - INFO - Trailer file written to:  astrodrizzle.log


Trailer file written to:  astrodrizzle.log


mv: rename /Users/avapolzin/Desktop/psf_testoutputs/acs/*.psf to psfs/*.psf: No such file or directory
mv: rename /Users/avapolzin/Desktop/psf_testoutputs/acs/*.cat to psfs/*.cat: No such file or directory
mv: rename /Users/avapolzin/Desktop/psf_testoutputs/acs/*_mask.fits to psfs/*_mask.fits: No such file or directory


In this case dpsf will be very simple, since we are only looking at one set of coordinates and one filter:

In [5]:
dpsf

{'10:00:33.0178 +02:09:52.304': {'F475W': array([[ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.]],
        shape=(2176, 4152), dtype='>f4')}}

By default, PSFs are generated for `spike.psf.hst` by `TinyTim` using a blackbody model of a G5V star at 6000 K, but users have complete control over the specifics of the PSF generation. To change the parameters within a method (e.g., updating model arguments, setting different detection thresholds for the empirical PSFs, ...), `spike.psf.hst` directly takes keyword arguments for specified 'method'. Details of allowed arguments are available in the `spike.psfgen` [documentation](https://spike-psf.readthedocs.io/en/latest/psfgen.html). As an example, we will change the `TinyTim` model inputs to use an O6 star at 45000 K:

In [6]:
dpsf = hst(img_dir = datapath, obj = obj, img_type = 'flc', inst = 'ACS', camera = 'WFC', savedir = outputpath, pretweaked = True, 
          listchoice = 'O6', temp = 45000)

/Users/avapolzin/opt/anaconda3/envs/spike/lib/python3.10/site-packages/spike/psf/psf.py:185: Warning: All of major, minor, and angle must be specified to be applied. Proceeding with no elliptical jitter.


Tiny Tim v7.5
Intermediate PSF dimensions are 464 by 464

Computing PSF for position 1/1 (x,y) = 1020 181
   Computing PSF 1/18 for wavelength 393.82 nm (weight=0.018692)
   Computing PSF 2/18 for wavelength 402.92 nm (weight=0.046730)
   Computing PSF 3/18 for wavelength 412.02 nm (weight=0.064174)
   Computing PSF 4/18 for wavelength 421.11 nm (weight=0.070042)
   Computing PSF 5/18 for wavelength 430.21 nm (weight=0.074734)
   Computing PSF 6/18 for wavelength 439.31 nm (weight=0.073655)
   Computing PSF 7/18 for wavelength 448.41 nm (weight=0.071159)
   Computing PSF 8/18 for wavelength 457.51 nm (weight=0.067712)
   Computing PSF 9/18 for wavelength 466.61 nm (weight=0.065917)
   Computing PSF 10/18 for wavelength 475.70 nm (weight=0.062852)
   Computing PSF 11/18 for wavelength 484.80 nm (weight=0.059882)
   Computing PSF 12/18 for wavelength 493.90 nm (weight=0.057730)
   Computing PSF 13/18 for wavelength 503.00 nm (weight=0.055124)
   Computing PSF 14/18 for wavelength 512.10 

/Users/avapolzin/opt/anaconda3/envs/spike/lib/python3.10/site-packages/spike/psf/psf.py:185: Warning: All of major, minor, and angle must be specified to be applied. Proceeding with no elliptical jitter.


Tiny Tim v7.5
Intermediate PSF dimensions are 464 by 464

Computing PSF for position 1/1 (x,y) = 1025 242
   Computing PSF 1/18 for wavelength 393.82 nm (weight=0.018692)
   Computing PSF 2/18 for wavelength 402.92 nm (weight=0.046730)
   Computing PSF 3/18 for wavelength 412.02 nm (weight=0.064174)
   Computing PSF 4/18 for wavelength 421.11 nm (weight=0.070042)
   Computing PSF 5/18 for wavelength 430.21 nm (weight=0.074734)
   Computing PSF 6/18 for wavelength 439.31 nm (weight=0.073655)
   Computing PSF 7/18 for wavelength 448.41 nm (weight=0.071159)
   Computing PSF 8/18 for wavelength 457.51 nm (weight=0.067712)
   Computing PSF 9/18 for wavelength 466.61 nm (weight=0.065917)
   Computing PSF 10/18 for wavelength 475.70 nm (weight=0.062852)
   Computing PSF 11/18 for wavelength 484.80 nm (weight=0.059882)
   Computing PSF 12/18 for wavelength 493.90 nm (weight=0.057730)
   Computing PSF 13/18 for wavelength 503.00 nm (weight=0.055124)
   Computing PSF 14/18 for wavelength 512.10 

/Users/avapolzin/opt/anaconda3/envs/spike/lib/python3.10/site-packages/spike/psf/psf.py:185: Warning: All of major, minor, and angle must be specified to be applied. Proceeding with no elliptical jitter.


...

                  Total          5.4554 sec.


2025-05-07 19:39:00,341 - stpipe - INFO - 


2025-05-07 19:39:00,346 - stpipe - INFO - Trailer file written to:  astrodrizzle.log


Trailer file written to:  astrodrizzle.log


mv: rename /Users/avapolzin/Desktop/psf_testoutputs/acs/*.psf to psfs/*.psf: No such file or directory
mv: rename /Users/avapolzin/Desktop/psf_testoutputs/acs/*.cat to psfs/*.cat: No such file or directory
mv: rename /Users/avapolzin/Desktop/psf_testoutputs/acs/*_mask.fits to psfs/*_mask.fits: No such file or directory


`spike` can also be used to generate PSFs for multiple objects/coordinates simultaneously.

In [4]:
obj = ['10:00:33.0178 +02:09:52.304', '10:00:28.1798 +02:08:40.507']

dpsf = hst(img_dir = datapath, obj = obj, img_type = 'flc', inst = 'ACS', camera = 'WFC', savedir = outputpath, pretweaked = True)

2025-05-07 20:31:49,191 - stpipe - WARNING - /Users/avapolzin/opt/anaconda3/envs/spike/lib/python3.10/site-packages/spike/psf/psf.py:236: Warning: All of major, minor, and angle must be specified to be applied. Proceeding with no elliptical jitter.



Tiny Tim v7.5
Intermediate PSF dimensions are 464 by 464

Computing PSF for position 1/1 (x,y) = 1020 181
   Computing PSF 1/18 for wavelength 393.82 nm (weight=0.008917)
   Computing PSF 2/18 for wavelength 402.92 nm (weight=0.024755)
   Computing PSF 3/18 for wavelength 412.02 nm (weight=0.037561)
   Computing PSF 4/18 for wavelength 421.11 nm (weight=0.045079)
   Computing PSF 5/18 for wavelength 430.21 nm (weight=0.052658)
   Computing PSF 6/18 for wavelength 439.31 nm (weight=0.056584)
   Computing PSF 7/18 for wavelength 448.41 nm (weight=0.059373)
   Computing PSF 8/18 for wavelength 457.51 nm (weight=0.061140)
   Computing PSF 9/18 for wavelength 466.61 nm (weight=0.064193)
   Computing PSF 10/18 for wavelength 475.70 nm (weight=0.065803)
   Computing PSF 11/18 for wavelength 484.80 nm (weight=0.067198)
   Computing PSF 12/18 for wavelength 493.90 nm (weight=0.069242)
   Computing PSF 13/18 for wavelength 503.00 nm (weight=0.070478)
   Computing PSF 14/18 for wavelength 512.10 

2025-05-07 20:31:52,399 - stpipe - WARNING - /Users/avapolzin/opt/anaconda3/envs/spike/lib/python3.10/site-packages/spike/psf/psf.py:236: Warning: All of major, minor, and angle must be specified to be applied. Proceeding with no elliptical jitter.



Tiny Tim v7.5
Processing PSF for position 1/1 : (x,y) = 1020 181
Reading input PSF from /Users/avapolzin/Desktop/psf_testoutputs/acs/j8pu42ecq_flc150d08m15.267s+2d09m52.304s_F475W_psf00_psf.fits.
  Input critically-sampled undistorted PSF dimensions are 464 by 464 (0.013018 arcsec/pixel).
  Mapping PSF onto distorted grid.
  Convolving PSF with charge diffusion kernel.
  Writing distorted PSF to /Users/avapolzin/Desktop/psf_testoutputs/acs/j8pu42ecq_flc150d08m15.267s+2d09m52.304s_F475W_psf00.fits (146 by 146 pixels)

Started at  Wed May  7 20:31:51 2025
Finished at Wed May  7 20:31:52 2025
Tiny Tim v7.5
Intermediate PSF dimensions are 464 by 464

Computing PSF for position 1/1 (x,y) = 2652 1305
   Computing PSF 1/18 for wavelength 393.82 nm (weight=0.008917)
   Computing PSF 2/18 for wavelength 402.92 nm (weight=0.024755)
   Computing PSF 3/18 for wavelength 412.02 nm (weight=0.037561)
   Computing PSF 4/18 for wavelength 421.11 nm (weight=0.045079)
   Computing PSF 5/18 for wavelength

2025-05-07 20:31:55,538 - stpipe - WARNING - /Users/avapolzin/opt/anaconda3/envs/spike/lib/python3.10/site-packages/spike/psf/psf.py:236: Warning: All of major, minor, and angle must be specified to be applied. Proceeding with no elliptical jitter.



...

                  Total          15.5602 sec.


2025-05-07 20:32:46,210 - stpipe - INFO - 


2025-05-07 20:32:46,219 - stpipe - INFO - Trailer file written to:  astrodrizzle.log


Trailer file written to:  astrodrizzle.log


Exception: No co-added/resampled output files created. Check your input path, coordinates and the output of the PSF generation steps.

In [5]:
dpsf

NameError: name 'dpsf' is not defined

One advantage of `spike` is that it can be used flexibly with a variety of different PSF generation methods -- to change which one is used, simply use the 'method' argument. Built-in options are:
- 'TinyTim’
- ‘TinyTim_Gillis’
- ‘STDPSF’
- ‘epsf’
- ‘PSFEx’

If `method = 'USER'`, then the keyword argument 'usermethod' must also be specified, pointing to either a function or a directory that houses model PSFs that follow the \[imgprefix]\_\[coords]\_\[band]\_psf.fits, e.g., imgprefix_23.31+30.12_F814W_psf.fits or imgprefix_195.78-46.52_F555W_psf.fits naming scheme.

We'll use STDPSFs below and have `spike` output cutouts around the PSFs.

`spike` can handle most plotting automatically and generates PSFs regardless of whether they are plotted. In most cases, `spike` is the only package you'll actually need to both create and visualize your PSFs.